In [1]:
import pandas as pd
import numpy as np
import sklearn
import xmltodict

In [2]:
import urllib2
import datetime

In [3]:
def read_data(filenames):
    arrays = [0,0,0]
    for i in xrange(len(filenames)):
        arrays[i] = pd.read_csv(filenames[i], parse_dates=['timestamp'])
    return arrays

In [4]:
def my_transport(data):
    data["metro_km_walk"]=data["metro_km_walk"].fillna(data["metro_km_avto"])
    data["metro_min_walk"]=data["metro_min_walk"].fillna(data["metro_km_avto"]*12)
    data["railroad_station_walk_km"]=data["railroad_station_walk_km"].fillna(data["railroad_station_avto_km"])
    data["railroad_station_walk_min"]=data["railroad_station_walk_min"].fillna(data["railroad_station_avto_km"]*12)
    data["ID_railroad_station_walk"]=data["ID_railroad_station_walk"].fillna(data["ID_railroad_station_avto"])
    return data

In [5]:
from sklearn.preprocessing import Imputer
def my_cafe(data):
    st0=['cafe_sum_5000_min_price_avg','cafe_sum_5000_max_price_avg','cafe_avg_price_5000']
    l=['5000','3000','2000','1500','1000','500']
    medImputer=Imputer(strategy='median')
    data[st0]=medImputer.fit_transform(data[st0])
    for i in xrange(len(l)-1):
        st1=[st.replace('5000',str(l[i+1])) for st in st0]
        st2=[st.replace('5000',str(l[i])) for st in st0]
        for k in xrange(len(st1)):
            data[st1[k]]=data[st1[k]].fillna(data[st2[k]])
    return data        

In [6]:
def er10(data):
    err = (data.full_sq>data.life_sq*10)&(data.life_sq>2)
    data.ix[err,'full_sq'] = data.ix[err,'full_sq']/10
    return data

In [7]:
def clean_sq(train):
    bad_index = train[(train.life_sq > train.full_sq) | 
                      (train.life_sq < 5)].index
    train.ix[bad_index, "life_sq"] = np.NaN

    bad_index = train[(train.full_sq > 210) & (train.life_sq / train.full_sq < 0.3) | 
                      (train.full_sq < 5)].index
    train.ix[bad_index, "full_sq"] = np.NaN
    
    bad_index = train[(train.kitch_sq >= train.life_sq) |
                      (train.kitch_sq == 0) | 
                      (train.kitch_sq == 1)].index
    train.ix[bad_index, "kitch_sq"] = np.NaN
    
    bad_index = train[train.life_sq > 300].index
    train.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN
    
    bad_index = train[train.build_year < 1500].index
    train.ix[bad_index, "build_year"] = np.NaN
    
    bad_index = train[train.num_room == 0].index 
    train.ix[bad_index, "num_room"] = np.NaN
    
    bad_index = train[train.max_floor == 0].index
    train.ix[bad_index, "max_floor"] = np.NaN
    
    bad_index = train[train.floor > train.max_floor].index
    train.ix[bad_index, "max_floor"] = np.NaN    
    
    return(train)    

In [8]:
def add_dates(train):
    
    # Add month-year
    month_year = (train.timestamp.dt.month + train.timestamp.dt.year * 100)
    month_year_cnt_map = month_year.value_counts().to_dict()
    train['month_year_cnt'] = month_year.map(month_year_cnt_map)
    
    # Add week-year count
    week_year = (train.timestamp.dt.weekofyear + train.timestamp.dt.year * 100)
    week_year_cnt_map = week_year.value_counts().to_dict()
    train['week_year_cnt'] = week_year.map(week_year_cnt_map)
    
    # Add month and day-of-week
    train['month'] = train.timestamp.dt.month
    train['dow'] = train.timestamp.dt.dayofweek
    
    return train
    

In [9]:
def new_features(train):
    
    train['rel_floor'] = train['floor'] / train['max_floor'].astype(float)
    train['rel_kitch_sq'] = train['kitch_sq'] / train['full_sq'].astype(float)
    train.apartment_name=train.sub_area + train['metro_km_avto'].astype(str)
    train['room_size'] = train['life_sq'] / train['num_room'].astype(float)
    
    return train

In [10]:
def labels(x_train):   
    for c in x_train.columns:
        if x_train[c].dtype == 'object':
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(x_train[c].values))
            x_train[c] = lbl.transform(list(x_train[c].values))
            #print c
            #x_train.drop(c,axis=1,inplace=True)    
    return x_train   

In [11]:
def printna(data):
    k=len(data)-data.count()
    print k[k>0]

In [12]:
def month_median(macro):
    macro["year"]  = macro["timestamp"].dt.year
    macro["month"] = macro["timestamp"].dt.month
    macro["yearmonth"] = 100*macro.year + macro.month
    macmeds = macro.groupby("yearmonth").median()
    return macmeds

In [13]:
import numpy.matlib as ml
 
def almonZmatrix(X, maxlag, maxdeg):
    """
    Creates the Z matrix corresponding to vector X.
    """
    n = len(X)
    Z = ml.zeros((len(X)-maxlag, maxdeg+1))
    for t in range(maxlag,  n):
       #Solve for Z[t][0].
       Z[t-maxlag,0] = sum([X[t-lag] for lag in range(maxlag+1)])
       for j in range(1, maxdeg+1):
             s = 0.0
             for i in range(1, maxlag+1):       
                s += (i)**j * X[t-i]
             Z[t-maxlag,j] = s
    return Z

In [14]:
def bad_address(data):
    data=data.set_index('id')
    fx = pd.read_excel('./data/BAD_ADDRESS_FIX.xlsx').drop_duplicates('id').set_index('id')
    data.update(fx)
    print('Fix: ', data.index.intersection(fx.index).shape[0])
    return data.reset_index()

In [15]:
import seaborn as sns
from sklearn import model_selection, preprocessing
import xgboost as xgb

/home/serzzh/anaconda2/envs/gl-env/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
# Parameters
micro_humility_factor = 1     #    range from 0 (complete humility) to 1 (no humility)
macro_humility_factor = 0.96
jason_weight = .2
bruno_weight = .2
reynaldo_weight = 1 - jason_weight - bruno_weight

In [17]:
filenames = ["./data/macro.csv","./data/train.csv/train.csv","./data/test.csv/test.csv"]
[macro, train, test] = read_data(filenames)
train, test = map(bad_address, [train, test])
[macro, train] = map(month_median,[macro, train])
id_test = test.id
df = macro.join(train, lsuffix='_left')

('Fix: ', 550)
('Fix: ', 149)


In [18]:
# Prepare data for macro model
import statsmodels.api as sm
y = df.price_doc.div(df.cpi).apply(np.log).loc[201108:201506]
lncpi = df.cpi.apply(np.log)
tblags = 5    # Number of lags used on PDL for Trade Balance
mrlags = 5    # Number of lags used on PDL for Mortgage Rate
cplags = 5    # Number of lags used on PDL for CPI
ztb = almonZmatrix(df.balance_trade.loc[201103:201506].as_matrix(), tblags, 1)
zmr = almonZmatrix(df.mortgage_rate.loc[201103:201506].as_matrix(), mrlags, 1)
zcp = almonZmatrix(lncpi.loc[201103:201506].as_matrix(), cplags, 1)
columns = ['tb0', 'tb1', 'mr0', 'mr1', 'cp0', 'cp1']
z = pd.DataFrame( np.concatenate( (ztb, zmr, zcp), axis=1), y.index.values, columns )
X = sm.add_constant( z )

# Fit macro model
eq = sm.OLS(y, X)
fit = eq.fit()

# Predict with macro model
test_cpi = df.cpi.loc[201507:201605]
test_index = test_cpi.index
ztb_test = almonZmatrix(df.balance_trade.loc[201502:201605].as_matrix(), tblags, 1)
zmr_test = almonZmatrix(df.mortgage_rate.loc[201502:201605].as_matrix(), mrlags, 1)
zcp_test = almonZmatrix(lncpi.loc[201502:201605].as_matrix(), cplags, 1)
z_test = pd.DataFrame( np.concatenate( (ztb_test, zmr_test, zcp_test), axis=1), 
                       test_index, columns )
X_test = sm.add_constant( z_test )
pred_lnrp = fit.predict( X_test )
pred_p = np.exp(pred_lnrp) * test_cpi

# Merge with test cases and compute mean for macro prediction
test["timestamp"] = pd.to_datetime(test["timestamp"])
test["year"]  = test["timestamp"].dt.year
test["month"] = test["timestamp"].dt.month
test["yearmonth"] = 100*test.year + test.month
test_ids = test[["yearmonth","id"]]
monthprices = pd.DataFrame({"yearmonth":pred_p.index.values,"monthprice":pred_p.values})
macro_mean = np.exp(test_ids.merge(monthprices, on="yearmonth").monthprice.apply(np.log).mean())
macro_mean

6593788.9177231779

In [19]:
# Naive macro model assumes housing prices will simply follow CPI
naive_pred_lnrp = y.mean()
naive_pred_p = np.exp(naive_pred_lnrp) * test_cpi
monthnaive = pd.DataFrame({"yearmonth":pred_p.index.values, "monthprice":naive_pred_p.values})
macro_naive = np.exp(test_ids.merge(monthnaive, on="yearmonth").monthprice.apply(np.log).mean())
macro_naive

7773440.7512487005

In [20]:
# Combine naive and substantive macro models
macro_mean = macro_naive * (macro_mean/macro_naive) ** macro_humility_factor
macro_mean

6637341.6089008758

In [21]:
filenames = ["./data/macro.csv","./data/train.csv/train.csv","./data/test.csv/test.csv"]
[macro, train, test] = read_data(filenames)
train, test = map(bad_address, [train, test])
train=er10(er10(train))
[train, test] = [my_transport(train),my_transport(test)]
[train, test] = [my_cafe(train),my_cafe(test)]
#data["pr_RUB_SQM"] = data["price_doc"]/data["full_sq"]
#data = excl_cheat(data)
id_test = test.id

('Fix: ', 550)
('Fix: ', 149)


In [22]:
#clean data
equal_index = [601,1896,2791]
test.ix[equal_index, "life_sq"] = test.ix[equal_index, "full_sq"]

kitch_is_build_year = [13117]
train.ix[kitch_is_build_year, "build_year"] = train.ix[kitch_is_build_year, "kitch_sq"]

bad_index = [10076, 11621, 17764, 19390, 24007, 26713, 29172]
train.ix[bad_index, "num_room"] = np.NaN

bad_index = [3174, 7313]
test.ix[bad_index, "num_room"] = np.NaN

[train, test] = [clean_sq(train), clean_sq(test)]

train.product_type.value_counts(normalize= True)
test.product_type.value_counts(normalize= True)


train.floor.describe(percentiles= [0.9999])
bad_index = [23584]
train.ix[bad_index, "floor"] = np.NaN
train.material.value_counts()
test.material.value_counts()
train.state.value_counts()
bad_index = train[train.state == 33].index
train.ix[bad_index, "state"] = np.NaN
test.state.value_counts()

2.0    2662
1.0    2266
3.0    1913
4.0     127
Name: state, dtype: int64

In [23]:
# brings error down a lot by removing extreme price per sqm
train.loc[train.full_sq == 0, 'full_sq'] = 50
train = train[train.price_doc/train.full_sq <= 600000]
train = train[train.price_doc/train.full_sq >= 10000]

In [24]:
# Other feature engineering
[train, test] = [add_dates(train), add_dates(test)]
train, test = new_features(train), new_features(test)

In [25]:
y_train = train["price_doc"]
x_train = train.drop(["id", "timestamp", "price_doc"], axis=1)
x_test = test.drop(["id", "timestamp"], axis=1)

x_train, x_test = labels(x_train), labels(x_test)

In [26]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=350)

y_predict = model.predict(dtest)
y_predict = np.round(y_predict * 0.99)
gunja_output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
gunja_output.head()

gunja_output.to_csv('gunja.csv', index=False)

In [27]:
[macro, train, test] = read_data(filenames)
train, test = map(bad_address, [train, test])
id_test = test.id

mult = .969

y_train = train["price_doc"] * mult + 10
x_train = train.drop(["id", "timestamp", "price_doc"], axis=1)
x_test = test.drop(["id", "timestamp"], axis=1)


x_train = labels(x_train)
x_test = labels(x_test)

xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train)
dtest = xgb.DMatrix(x_test)

num_boost_rounds = 384  # This was the CV output, as earlier version shows
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round= num_boost_rounds)

y_predict = model.predict(dtest)
output = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
output.head()

('Fix: ', 550)
('Fix: ', 149)


,id,price_doc
0,30474,5153489.5
1,30475,8215832.0
2,30476,5350957.5
3,30477,5729383.0
4,30478,4959933.0


In [28]:
[df_macro, df_train, df_test] = read_data(filenames)
train, test = map(bad_address, [train, test])

df_train.drop(df_train[df_train["life_sq"] > 7000].index, inplace=True)

y_train = df_train['price_doc'].values  * mult + 10
id_test = df_test['id']

df_train.drop(['id', 'price_doc'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)

num_train = len(df_train)
df_all = pd.concat([df_train, df_test])
# Next line just adds a lot of NA columns (becuase "join" only works on indexes)
# but somewhow it seems to affect the result
df_all = df_all.join(df_macro, on='timestamp', rsuffix='_macro')
print(df_all.shape)


df_all = add_dates(df_all)

df_all = new_features(df_all)


# Remove timestamp column (may overfit the model in train)
df_all.drop(['timestamp', 'timestamp_macro'], axis=1, inplace=True)


factorize = lambda t: pd.factorize(t[1])[0]

df_obj = df_all.select_dtypes(include=['object'])

X_all = np.c_[
    df_all.select_dtypes(exclude=['object']).values,
    np.array(list(map(factorize, df_obj.iteritems()))).T
]
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]


# Deal with categorical values
df_numeric = df_all.select_dtypes(exclude=['object'])
df_obj = df_all.select_dtypes(include=['object']).copy()

for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]

df_values = pd.concat([df_numeric, df_obj], axis=1)


# Convert to numpy values
X_all = df_values.values
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]

df_columns = df_values.columns


xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(X_train, y_train, feature_names=df_columns)
dtest = xgb.DMatrix(X_test, feature_names=df_columns)


num_boost_round = 489  # From Bruno's original CV, I think
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_round)

y_pred = model.predict(dtest)

df_sub = pd.DataFrame({'id': id_test, 'price_doc': y_pred})

df_sub.head()

('Fix: ', 550)
('Fix: ', 149)
(38132, 390)
(38132, 395)
(38132, 395)


,id,price_doc
0,30474,5254606.5
1,30475,8110713.0
2,30476,5363647.5
3,30477,5517785.5
4,30478,4974669.5


In [29]:
first_result = output.merge(df_sub, on="id", suffixes=['_louis','_bruno'])
first_result["price_doc"] = np.exp( .714*np.log(first_result.price_doc_louis) + 
                                    .286*np.log(first_result.price_doc_bruno) )  # multiplies out to .5 & .2
result = first_result.merge(gunja_output, on="id", suffixes=['_follow','_gunja'])

result["price_doc"] = np.exp( .7*np.log(result.price_doc_follow) + 
                              .3*np.log(result.price_doc_gunja) )
result.drop(["price_doc_louis","price_doc_bruno","price_doc_follow","price_doc_gunja"],axis=1,inplace=True)
result.head()
result.to_csv('unadj_sub.csv', index=False)

In [30]:
# Adjust

lny = np.log(result.price_doc)
lnm = np.log(macro_mean)

# I'm not sure whether this makes any sense or not.
# 1+lny.mean()-lnm term is meant to offest the scale effect of the logarithmic mean shift
#   while allowing the new logarithmic mean to remain at lnm.
y_trans = lnm  +  micro_humility_factor * (lny-lny.mean()) * (1+lny.mean()-lnm)
y_predict = np.exp( y_trans )

sub = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
sub.head()
sub.to_csv('sub.csv', index=False)